<https://github.com/PolymathicAI/xVal>


In [1]:
from flax import nnx
import flax

flax.__version__

'0.10.2'

In [2]:
import ast
import re
from datetime import datetime as dt
import os

import icecream
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import optax
from flax.struct import dataclass
from icecream import ic
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizerFast, FlaxBertModel

import hephaestus as hp

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)
pd.options.mode.copy_on_write = True

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: {('pooler', 'dense', 'kernel'), ('pooler', 'dense', 'bias')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(3, 768)

In [5]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [6]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")


# Combine total mass of all planets into one column `planet<n>_m`
mass_regex = re.compile(r"planet(\d+)_m")
mass_cols = [col for col in df.columns if mass_regex.match(col)]
df["total_mass"] = df[mass_cols].sum(axis=1)
# df = df.reset_index(drop=True)
# Introduce categorical columns for the number of planets choose non null columns with mass
df["n_planets"] = df[mass_cols].notnull().sum(axis=1).astype("object")
df["n_planets"] = df["n_planets"].apply(lambda x: f"{x}_planets")
# Create category acceleration if the sum of plane/d_[x,y, z] is greater than 0
df["acceleration_x"] = df[
    [col for col in df.columns if "planet" in col and "_x" in col]
].sum(axis=1)
# Set acceleration_x to "increasing" if greater than 0 else "decreasing"
df["acceleration_x"] = (
    df["acceleration_x"]
    .apply(lambda x: "increasing" if x > 0 else "decreasing")
    .astype("object")
)
df["acceleration_y"] = df[
    [col for col in df.columns if "planet" in col and "_y" in col]
].sum(axis=1)
df["acceleration_y"] = df["acceleration_y"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)


df.describe()

,idx,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
count,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,4.165044e+06,4.165044e+06,5.563957e+06,5.563957e+06,...,2.783627e+06,2.783627e+06,2.783627e+06,2.783627e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,5.563957e+06
mean,6.248635e+04,9.748911e+00,-1.339198e-01,7.391138e-02,-1.340140e-01,7.291389e-02,-1.305344e-01,7.065633e-02,2.999306e+00,1.624756e+00,...,6.559150e-02,2.996303e+00,1.623874e+00,9.980576e-01,-1.276881e-01,6.519469e-02,3.002531e+00,1.625815e+00,1.001317e+00,1.049149e+01
std,3.607949e+04,5.993534e+00,1.228071e+00,1.213232e+00,1.227950e+00,1.212650e+00,1.217229e+00,1.203678e+00,1.157182e+00,5.876632e-01,...,1.200148e+00,1.153190e+00,5.270725e-01,5.764675e-01,1.211648e+00,1.199625e+00,1.156856e+00,5.167198e-01,5.779763e-01,3.991780e+00
min,0.000000e+00,0.000000e+00,-3.294763e+00,-2.997514e+00,-3.284004e+00,-2.998546e+00,-3.289790e+00,-2.998050e+00,1.000003e+00,1.000000e+00,...,-2.997621e+00,1.000054e+00,1.000000e+00,9.369537e-05,-3.273603e+00,-2.998913e+00,1.000103e+00,1.000000e+00,6.720938e-05,2.014597e+00
25%,3.124400e+04,4.655172e+00,-1.030131e+00,-9.020907e-01,-1.030516e+00,-9.028009e-01,-1.050662e+00,-9.211662e-01,1.993948e+00,1.000000e+00,...,-9.321272e-01,1.996853e+00,1.191548e+00,4.980967e-01,-1.071974e+00,-9.394428e-01,2.004240e+00,1.215927e+00,5.032645e-01,7.282371e+00
50%,6.249100e+04,9.523810e+00,-1.542335e-01,1.117099e-01,-1.538916e-01,1.099474e-01,-1.525520e-01,1.118031e-01,2.994477e+00,1.543047e+00,...,1.067355e-01,3.001879e+00,1.535683e+00,9.955040e-01,-1.507184e-01,1.055276e-01,3.000454e+00,1.519040e+00,1.003118e+00,1.028205e+01
75%,9.372800e+04,1.440000e+01,8.583344e-01,9.784987e-01,8.581045e-01,9.783998e-01,8.762358e-01,9.912902e-01,4.005747e+00,2.020672e+00,...,9.956064e-01,3.986406e+00,1.969644e+00,1.497746e+00,8.906780e-01,9.991220e-01,4.003332e+00,1.950518e+00,1.503457e+00,1.345633e+01
max,1.249990e+05,2.400000e+01,2.996370e+00,2.999014e+00,2.993319e+00,2.999536e+00,2.990464e+00,2.998478e+00,4.999994e+00,2.999984e+00,...,3.000881e+00,4.999990e+00,2.999909e+00,1.999957e+00,2.985180e+00,2.998936e+00,4.999679e+00,2.999497e+00,1.999999e+00,2.382455e+01


In [7]:
df.head()

,idx,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,...,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass,n_planets,acceleration_x,acceleration_y
0,0,0.000000,1.560060,-0.854437,0.720639,0.691729,0.944008,2.700632,1.312562,1.944263,...,NaN,NaN,NaN,NaN,NaN,NaN,6.974122,3_planets,increasing,increasing
1,0,0.465116,1.689858,-0.514359,0.333295,0.942289,0.681604,2.785811,1.312562,1.944263,...,NaN,NaN,NaN,NaN,NaN,NaN,6.974122,3_planets,increasing,increasing
2,0,0.930233,1.753589,-0.154209,-0.124995,0.992368,0.412951,2.845461,1.312562,1.944263,...,NaN,NaN,NaN,NaN,NaN,NaN,6.974122,3_planets,increasing,increasing
3,0,1.395349,1.748068,0.212022,-0.556775,0.831727,0.140540,2.879232,1.312562,1.944263,...,NaN,NaN,NaN,NaN,NaN,NaN,6.974122,3_planets,increasing,increasing
4,0,1.860465,1.673573,0.569904,-0.870579,0.494812,-0.133144,2.887018,1.312562,1.944263,...,NaN,NaN,NaN,NaN,NaN,NaN,6.974122,3_planets,increasing,increasing


In [8]:
df_categorical = df.select_dtypes(include=["object"]).astype(str)
unique_values_per_column = df_categorical.apply(
    pd.Series.unique
).values  # .flatten().tolist()
flattened_unique_values = np.concatenate(unique_values_per_column).tolist()
unique_values = list(set(flattened_unique_values))
unique_values

['3_planets',
 '2_planets',
 'decreasing',
 '4_planets',
 '5_planets',
 'increasing']

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [10]:
tokenizer.vocab_size

30522

In [11]:
df.select_dtypes(include="object").groupby(
    df.select_dtypes(include="object").columns.tolist()
).size().reset_index(name="count")

,n_planets,acceleration_x,acceleration_y,count
0,2_planets,decreasing,decreasing,365720
1,2_planets,decreasing,increasing,410992
2,2_planets,increasing,decreasing,272240
3,2_planets,increasing,increasing,349961
4,3_planets,decreasing,decreasing,410092
5,3_planets,decreasing,increasing,404513
6,3_planets,increasing,decreasing,258536
7,3_planets,increasing,increasing,308276
8,4_planets,decreasing,decreasing,413451
9,4_planets,decreasing,increasing,416444


In [12]:
df = df.reset_index(drop=True)

In [13]:
# Get train test split at 80/20
time_series_config = hp.TimeSeriesConfig.generate(df=df)
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = hp.TimeSeriesDS(train_df, time_series_config)
test_ds = hp.TimeSeriesDS(test_df, time_series_config)
len(train_ds), len(test_ds)

(99999, 25001)

In [14]:
len(time_series_config.numeric_col_tokens) + len(
    time_series_config.categorical_col_tokens
)

30

In [15]:
def make_batch(ds: hp.TimeSeriesDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# print(batch1["numeric"].shape, batch1["categorical"].shape)
# (4, 27, 59) (4, 3, 59)
# batch

In [16]:
multiplier = 4
time_series_regressor = hp.TimeSeriesDecoder(
    time_series_config, d_model=512, n_heads=8 * multiplier, rngs=nnx.Rngs(0)
)
# nnx.display(time_series_regressor)

In [17]:
embed = nnx.Embed(
    num_embeddings=time_series_config.n_tokens, features=512, rngs=nnx.Rngs(0)
)
# indices_input = jnp.array([[0, 1, 2], [-1, -2, -3]])
indices_input = jnp.array([0, 1])
embed(indices_input)

Array([[ 0.04437031,  0.04695525,  0.02283594, ..., -0.01555218,
        -0.00709071,  0.06242825],
       [ 0.00662031, -0.01167173, -0.02226578, ..., -0.01077074,
         0.01556168,  0.02268689]], dtype=float32)

In [18]:
token_reservoir_lookup = time_series_config.reservoir_encoded[1]
embed(token_reservoir_lookup)

Array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 0.04437031,  0.04695525,  0.02283594, ..., -0.01555218,
        -0.00709071,  0.06242825],
       [ 0.04437031,  0.04695525,  0.02283594, ..., -0.01555218,
        -0.00709071,  0.06242825]], dtype=float32)

In [19]:
token_reservoir_lookup

Array([ 1031, 16371, 25531,  1035,  7308,  1033,     0,     0], dtype=int32)

In [20]:
time_series_regressor.time_series_transformer.embedding(jnp.array([-1, -2, -3]))

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:420 in __call__()
    "Reservoir indices": 'Reservoir indices'
    jnp.isnan(reservoir_indices).any(): Array(False, dtype=bool)
    reservoir_indices.dtype: dtype('int32')
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:426 in __call__()
    "Return embed": 'Return embed'
    jnp.isnan(return_embed).any(): Array(False, dtype=bool)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:428 in __call__()
    "Return embed final": 'Return embed final'
    jnp.isnan(return_embed).any(): Array(False, dtype=bool)


Array([[ 0.20445955,  0.11651482, -0.14613178, ..., -0.30445826,
         0.13163236, -0.02010614],
       [ 0.1159903 ,  0.1296331 , -0.04656842, ..., -0.36805886,
         0.06944571,  0.11317717],
       [ 0.11495014,  0.05041841, -0.16187619, ..., -0.09441441,
         0.07160322, -0.03441812]], dtype=float32)

In [21]:
res = time_series_regressor(
    numeric_inputs=batch["numeric"],
    categorical_inputs=batch["categorical"],
    deterministic=False,
)

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:542 in process_numeric()
    "Before embedding": 'Before embedding'
    jnp.isnan(repeated_numeric_indices).any(): Array(False, dtype=bool)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:420 in __call__()
    "Reservoir indices": 'Reservoir indices'
    jnp.isnan(reservoir_indices).any(): Array(False, dtype=bool)
    reservoir_indices.dtype: dtype('int32')
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:426 in __call__()
    "Return embed": 'Return embed'
    jnp.isnan(return_embed).any(): Array(False, dtype=bool)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:428 in __call__()
    "Return embed final": 'Return embed final'
    jnp.isnan(return_embed).any(): Array(False, dtype=bool)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:546 in process_numeric()
    "After embedding": 'After embedding'
    jnp.i

In [22]:
res["numeric_out"].shape, res["categorical_out"].shape

((4, 27, 59), (4, 3, 59, 41))

In [23]:
if jnp.isnan(res["numeric_out"]).any():
    raise ValueError("NaN in numeric_out")
if jnp.isnan(res["categorical_out"]).any():
    raise ValueError("NaN in categorical_out")

In [24]:
ic.disable()

In [25]:
causal_mask = True
# time_series_regressor.train()

In [26]:
def add_input_offsets(
    inputs: jnp.array, outputs: jnp.array, inputs_offset: int = 1
) -> jnp.array:
    inputs = inputs[:, :, inputs_offset:]
    tmp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, tmp_null], axis=2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)
    # Add ext
    if outputs.ndim == inputs.ndim + 1:
        nan_mask_expanded = jnp.expand_dims(nan_mask, axis=-1)
        nan_mask_expanded = jnp.broadcast_to(nan_mask_expanded, outputs.shape)
    else:
        nan_mask_expanded = nan_mask

    # Apply mask to outputs
    outputs = jnp.where(nan_mask_expanded, jnp.zeros_like(outputs), outputs)

    return inputs, outputs, nan_mask
    # outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    # return inputs, outputs, nan_mask


def numeric_loss(inputs, outputs, input_offset: int = 1):
    # print("Doing Numeric Loss")
    inputs, outputs, nan_mask = add_input_offsets(
        inputs=inputs, outputs=outputs, inputs_offset=input_offset
    )
    # print(f"Numeric: {inputs.shape=}, {outputs.shape=}, {nan_mask.shape=}")
    # TODO make loss SSL for values greater than 0.5 and MSE for values less than 0.5
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    # print(f"{masked_loss.shape=}, {nan_mask.shape=}")
    loss = masked_loss.sum() / (~nan_mask).sum()
    return loss


def categorical_loss(inputs, outputs, input_offset: int = 1):
    # print("Doing Categorical Loss")
    inputs, outputs, nan_mask = add_input_offsets(
        inputs=inputs, outputs=outputs, inputs_offset=input_offset
    )
    # print(f"Categorical: {inputs.shape=}, {outputs.shape=}, {nan_mask.shape=}")
    inputs = inputs.astype(jnp.int32)
    # print(f"Categorical Loss Shapes: {inputs.shape=}, {outputs.shape=}")
    raw_loss = optax.softmax_cross_entropy_with_integer_labels(outputs, inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss).mean()
    return masked_loss


@nnx.jit
def train_step(
    model: hp.TimeSeriesDecoder,
    inputs: dict,
    optimizer: nnx.Optimizer,
    metrics: nnx.MultiMetric,
):
    def loss_fn(model):
        res = model(
            numeric_inputs=inputs["numeric"],
            categorical_inputs=inputs["categorical"],
            deterministic=False,
        )

        numeric_loss_value = numeric_loss(inputs["numeric"], res["numeric_out"])
        categorical_loss_value = categorical_loss(
            inputs["categorical"], res["categorical_out"]
        )
        loss = numeric_loss_value + categorical_loss_value
        return loss, (numeric_loss_value, categorical_loss_value)

    grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
    # grad_fn = nnx.value_and_grad(loss_fn, has_aux=False)
    # loss, grads = grad_fn(model)
    (loss, (numeric_loss_value, categorical_loss_value)), grads = grad_fn(model)
    metrics.update(
        loss=loss,
        numeric_loss=numeric_loss_value,
        categorical_loss=categorical_loss_value,
    )

    optimizer.update(grads)


metric_history = {
    "loss": [],
    "numeric_loss": [],
    "categorical_loss": [],
}

learning_rate = 1e-3
momentum = 0.9
optimizer = nnx.Optimizer(time_series_regressor, optax.adamw(learning_rate, momentum))
metrics = nnx.MultiMetric(
    loss=nnx.metrics.Average("loss"),
    categorical_loss=nnx.metrics.Average("categorical_loss"),
    numeric_loss=nnx.metrics.Average("numeric_loss"),
)
writer_name = "DoesThisWork"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
summary_writer = SummaryWriter("runs/" + model_name)


metrics_history = {"train_loss": []}
train_data_loader = DataLoader(train_ds, batch_size=16, shuffle=True)

for step, batch in enumerate(tqdm(train_data_loader)):
    batch = {"numeric": jnp.array(batch[0]), "categorical": jnp.array(batch[1])}
    train_step(time_series_regressor, batch, optimizer, metrics)
    for metric, value in metrics.compute().items():
        # Only shows `loss`

        metric_history[metric].append(value)
        if jnp.isnan(value).any():
            raise ValueError("Nan Values")
        summary_writer.add_scalar(f"train/{metric}", np.array(value), step)
    metrics.reset()
    # print(f"Step {step}")

  0%|          | 0/6250 [00:00<?, ?it/s]

No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values
No Nan Values


KeyboardInterrupt: 

In [ ]:
# import orbax.checkpoint as ocp

# ckpt_dir = ocp.test_utils.erase_and_create_empty("/tmp/my-checkpoints1/")
# _, state = nnx.split(time_series_regressor)
# state = state.to_pure_dict()
# # nnx.display(state)

# checkpointer = ocp.StandardCheckpointer()
# checkpointer.save(ckpt_dir / "state", state)

In [ ]:
# import jax.numpy as jnp
# import jax.tree_util as jtu


# # Print the structure and dtypes
# def print_dtype_tree(tree):
#     def get_dtype(x):
#         if hasattr(x, "dtype"):
#             return x.dtype
#         return type(x)

#     return jtu.tree_map(get_dtype, tree)


# print("State structure and dtypes:")
# print(jtu.tree_map(lambda x: type(x), state))
# print("\nDetailed dtype information:")
# print(print_dtype_tree(state))

In [27]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(model, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    out = time_series_regressor(
        numeric_inputs=numeric_inputs, categorical_inputs=categorical_inputs
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(time_series_regressor, train_ds, 0)
x.categorical_out.shape

(1, 3, 59, 41)

In [28]:
causal_mask = False
causal_mask = True


def process_results(arr: jnp.array, col_names: list, config: hp.TimeSeriesConfig):
    arr = jnp.squeeze(arr)
    if arr.ndim == 3:
        # Check if there is a logit array for example if there are 3 dims then the
        # last dim is the logit array. We need to get the argmax of the last dim
        # to get the actual values of the array and replace the logit array with the
        # actual values
        arr = jnp.argmax(arr, axis=-1)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


@dataclass
class DFComparison:
    input_df: pd.DataFrame
    output_df: pd.DataFrame


def show_results_df(
    model, time_series_config, dataset, idx: int = 0, mask_start: int = None
):
    results = return_results(model, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    input_numeric = process_results(
        results.numeric_inputs,
        time_series_config.numeric_col_tokens,
        time_series_config,
    )
    output_categorical = process_results(
        results.categorical_out,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    output_numeric = process_results(
        results.numeric_out, time_series_config.numeric_col_tokens, time_series_config
    )
    input_df = pd.concat([input_categorical, input_numeric], axis=1)
    output_df = pd.concat([output_categorical, output_numeric], axis=1)

    return DFComparison(input_df, output_df)


df_comp = show_results_df(
    model=time_series_regressor,
    time_series_config=time_series_config,
    dataset=train_ds,
    idx=0,
)

In [29]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

,n_planets,acceleration_x,acceleration_y
54,35,37,37
55,35,37,37
56,35,37,37
57,35,37,34
58,35,37,37


In [30]:
df_comp.output_df.loc[:, time_series_config.numeric_col_tokens].tail()

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
54,9.672389,0.613339,-0.177632,-0.600763,0.855175,0.324767,-1.249124,1.869614,2.002172,1.786154,...,3.470700,1.891295,-0.806448,1.874905,-2.062770,1.459731,3.970012,4.251047,1.143389,11.532623
55,9.475935,0.304420,-0.221440,-0.290383,1.164332,0.492031,-0.894148,1.896323,2.244848,1.756954,...,3.456095,2.025777,-0.730016,1.728912,-1.964452,1.700251,4.004183,4.385352,1.173101,11.513236
56,9.453676,0.230225,-0.326604,-0.276866,1.161639,0.509732,-1.079175,1.932898,2.202291,1.626413,...,3.476095,2.091459,-0.659264,1.645119,-1.895032,1.523275,4.015066,4.190773,1.166027,11.120915
57,9.413525,0.286363,-0.268869,-0.323321,1.234497,0.499288,-1.119666,1.884268,2.166392,1.707486,...,3.480995,1.898307,-0.649197,1.707443,-1.977850,1.614375,3.991729,4.274964,1.192942,11.276724
58,9.367428,0.388932,-0.090218,-0.414004,1.054981,0.468412,-1.225461,1.909914,2.133990,1.658625,...,3.513988,1.958603,-0.702737,1.787257,-1.913991,1.570186,3.757698,4.292969,1.019403,11.256743


In [31]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [32]:
jnp.array([True, True, False, False, True]).shape

(5,)

In [33]:
x = jnp.ones((5, 20))
print(x.shape)
xx = x.at[jnp.array([True, True, False, False, True]), :].set(0)
xx

(5, 20)


Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [68]:
@dataclass
class AutoRegressiveResults:
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array

    @classmethod
    def from_ds(cls, ds: hp.TimeSeriesDS, idx: int, stop_idx: int = 10):
        inputs = ds[idx]
        numeric_inputs = inputs[0][:, :stop_idx]
        categorical_inputs = inputs[1][:, :stop_idx]
        return cls(numeric_inputs, categorical_inputs)


def auto_regressive_predictions(
    model: hp.TimeSeriesDecoder,
    inputs: AutoRegressiveResults,
) -> jnp.array:
    numeric_inputs = inputs.numeric_inputs
    categorical_inputs = inputs.categorical_inputs

    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    # numeric_inputs = inputs.numeric_inputs
    # categorical_inputs = inputs.categorical_inputs
    numeric_nan_columns = jnp.isnan(numeric_inputs).all(axis=1)

    categorical_nan_columns = jnp.isnan(categorical_inputs).all(axis=1)

    # Expand first dim to make it a batch
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])

    outputs = time_series_regressor(
        numeric_inputs=numeric_inputs, categorical_inputs=categorical_inputs
    )
    numeric_out = outputs["numeric_out"]  #  jnp.squeeze(outputs["numeric_out"])
    # print(f"{numeric_out.shape=}")
    categorical_out = outputs[
        "categorical_out"
    ]  # jnp.squeeze(outputs["categorical_out"])

    categorical_out = jnp.argmax(categorical_out, axis=-1)

    final_numeric_row = np.array(numeric_out[:, :, -1])

    final_numeric_row = final_numeric_row[None, :]  # New axis
    final_numeric_row = jnp.transpose(final_numeric_row, (0, 2, 1))
    # print(f"{final_numeric_row.shape=}")
    final_categorical_row = np.array(categorical_out[:, :, -1])
    final_categorical_row = final_categorical_row[:, None]  # New axis
    final_categorical_row = jnp.transpose(final_categorical_row, (0, 2, 1))
    # print(f"{numeric_inputs.shape=}, {final_numeric_row.shape=}")
    numeric_inputs = jnp.concatenate(
        [numeric_inputs, final_numeric_row], axis=2
    )  # here

    categorical_inputs = jnp.concatenate(
        [categorical_inputs, final_categorical_row], axis=2
    )
    numeric_inputs = numeric_inputs.at[jnp.array(numeric_nan_columns)[None, :]].set(
        jnp.nan
    )
    categorical_inputs = categorical_inputs.at[
        jnp.array(categorical_nan_columns)[None, :]
    ].set(jnp.nan)
    inputs = (numeric_inputs, categorical_inputs)

    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)


test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)
# inputs_test = train_ds[0]
# test_numeric = inputs_test[0]
# test_categorical = inputs_test[1]
# print(inputs_test.shape)
for i in trange(21):
    print(f"Step {i}")
    inputs_test = auto_regressive_predictions(time_series_regressor, test_inputs)

# x = auto_regressive_predictions(state, test_ds[0], 10)

  0%|          | 0/21 [00:00<?, ?it/s]

Step 0
Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16
Step 17
Step 18
Step 19
Step 20


In [69]:
inputs_test

(Array([[[ 0.        ,  0.4651163 ,  0.9302326 ,  1.3953488 ,
           1.8604652 ,  2.3255813 ,  2.7906976 ,  3.2558138 ,
           3.7209303 ,  4.1860466 ,  9.482002  ],
         [ 1.5600603 ,  1.689858  ,  1.7535888 ,  1.748068  ,
           1.6735727 ,  1.533811  ,  1.3355262 ,  1.0878245 ,
           0.8013646 ,  0.48754898,  0.7794335 ],
         [-0.854437  , -0.5143588 , -0.15420859,  0.2120217 ,
           0.5699039 ,  0.9056069 ,  1.2068323 ,  1.4635128 ,
           1.6682035 ,  1.8161682 , -0.01436893],
         [ 0.7206392 ,  0.33329472, -0.12499522, -0.55677533,
          -0.8705794 , -1.0001508 , -0.9181916 , -0.6419799 ,
          -0.22980753,  0.23109236, -0.44991815],
         [ 0.69172925,  0.942289  ,  0.9923679 ,  0.83172745,
           0.49481156,  0.05317765, -0.39968765, -0.76795757,
          -0.97351485, -0.9724492 ,  0.8320009 ],
         [ 0.94400775,  0.68160415,  0.4129514 ,  0.14053972,
          -0.13314363, -0.4056377 , -0.6745295 , -0.93747383,
      

In [62]:
test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)

test_inputs.numeric_inputs.shape

(27, 10)

In [63]:
[train_ds[0][0][:, :10].shape, train_ds[0][1][:, :10].shape]

[(27, 10), (3, 10)]

In [64]:
time_series_regressor(
    numeric_inputs=jnp.array([train_ds[0][0]]),
    # numeric_inputs=jnp.array([train_ds[0][0][:, :10]]),
    categorical_inputs=jnp.array([train_ds[0][1]]),
    # categorical_inputs=jnp.array([train_ds[0][1][:, :10]]),
)

{'numeric_out': Array([[[ 9.039447  ,  8.960604  ,  8.880531  , ...,  9.584735  ,
           9.3857355 ,  9.3926935 ],
         [ 0.8240336 ,  0.75855994,  0.62180233, ...,  0.3898611 ,
           0.44426185,  0.3483044 ],
         [ 0.05398484, -0.0666422 , -0.10145084, ..., -0.18008521,
          -0.31192902, -0.13200954],
         ...,
         [ 3.9737306 ,  3.9629052 ,  3.9465425 , ...,  4.2801166 ,
           4.235305  ,  4.304021  ],
         [ 1.2193893 ,  1.204178  ,  1.1579299 , ...,  1.0548034 ,
           1.1442685 ,  1.0121615 ],
         [10.5649    , 10.631219  , 10.796641  , ..., 11.464384  ,
          11.23075   , 11.23721   ]]], dtype=float32),
 'categorical_out': Array([[[[ 0.73896015,  0.59590137,  0.00253134, ...,  0.00253134,
            1.1798776 ,  0.3913201 ],
          [ 1.2261488 ,  0.8438818 ,  0.00253134, ...,  0.00253134,
            0.9859587 ,  0.4030037 ],
          [ 0.6161045 ,  0.7935498 ,  0.00253134, ...,  0.00253134,
            0.3515788 ,  0.428

In [65]:
res1 = time_series_regressor(
    numeric_inputs=jnp.array([train_ds[0][0][:, :10]]),
    categorical_inputs=jnp.array([train_ds[0][1][:, :10]]),
)
res1

{'numeric_out': Array([[[ 8.90138245e+00,  9.01430702e+00,  8.92045116e+00,
           8.97206783e+00,  9.03429317e+00,  8.89560795e+00,
           9.17222691e+00,  9.35558319e+00,  9.34484577e+00,
           9.63436699e+00],
         [ 8.15452158e-01,  7.56613493e-01,  8.07569683e-01,
           7.35458970e-01,  6.47504687e-01,  6.61483467e-01,
           8.32620442e-01,  5.85060954e-01,  4.97587651e-01,
           4.75583494e-01],
         [ 1.12991575e-02,  9.43228230e-02,  6.99199662e-02,
           5.74195608e-02, -5.03006950e-02,  7.44459257e-02,
          -3.78151909e-02, -1.16956748e-01, -5.21951094e-02,
          -1.14257492e-01],
         [-4.18817163e-01, -5.15254855e-01, -4.29645658e-01,
          -4.75570560e-01, -4.70824599e-01, -3.74825358e-01,
          -5.52993655e-01, -3.46581101e-01, -4.31349635e-01,
          -5.75156569e-01],
         [ 8.73727918e-01,  7.23250031e-01,  7.02519059e-01,
           7.59190381e-01,  6.04880929e-01,  8.45112920e-01,
           7.751618

In [66]:
jnp.isnan(res1["numeric_out"]).any()

Array(False, dtype=bool)

In [67]:
test_inputs.numeric_inputs.shape

(27, 10)